In [ ]:
import os
import requests
import zipfile
import geopandas
import pandas
from osgeo import gdal

# Set working directory
Please change the settings for your own

In [ ]:
#change !!!
working_dir = "D:/FabianHDD/Uni_Master/modulare Programmierung/dataB3"
# shape for the aoi
shp="D:/FabianHDD/Uni_Master/modulare Programmierung/documentsB/AOI.shp"
# file for raster correction. Due to license rights you have to get the data by your own.
file_cor_dgm = "D:/FabianHDD/Uni_Master/modulare Programmierung/documentsB/GCG2016.tif"



os.chdir(working_dir)
year = 2017
zip_files_to_delete = []

# Create variable list needed for dowload of elevation data

In [ ]:
def set_elev_variables(year):
    # set the variables if there could be data available for this year
    if 2011 <= year <= 2013:
        url_year = "2010-2013"
        dem_n = "2"
        elev_meta_file = "DGM2_2010-2013_Erfass-lt-Meta_UTM32-UTM_2014-12-10.shp"
    elif 2014 <= year <= 2019:
        url_year = "2014-2019"
        dem_n = "1"
        elev_meta_file = "DGM1_2014-2019_Erfass-lt-Meta_UTM_2020-04-20--17127.shp"
    elif 2020 <= year <= 2025:
        url_year = "2020-2025"
        dem_n = "1"
        elev_meta_file = "DGM1_2020-2025_Erfass-lt-Meta_UTM_2021-03--17127/" \
                         "DGM1_2020-2025_Erfass-lt-Meta_UTM_2021-03--17127.shp"
    # return stop if there is certainly no data available for this year
    else:
        return "stop", "stop", "stop"
    # return the defined variables
    return url_year, dem_n, elev_meta_file

url_year, dem_n, elev_meta_file = set_elev_variables(year)
print(elev_meta_file)

# Dowload metadata shapefile

In [ ]:
def data_download(type_to_download, data_list_to_download, url_year="", year=0, dem_n="", year_list=None,
                  tile_number_list=None):
    # the two following variables are used for the naming of the zip files (data kind = dgm, dom or las)
    data_year = "_" + str(year)
    data_kind = ""
    # in this list the names of the zip files are stored
    zip_data_list = []
    # necessary for the orthophoto lists
    index = 0
    # for loop to download more than one file
    for i in data_list_to_download:
        # set the url, the one or two of the variables  used for the naming of the zip files and
        # get the name of the zip file content and its hypothetically file path
        # to be able to check if the file is  already present
        url = ""
        hy_file_path = """elevation_data/{}/{}/""".format(type_to_download, str(year))
        file_name = ""
        file_name_part_1 = ""
        file_name_part_2 = ""
        if type_to_download == "meta_data" or type_to_download == "auxiliary_data":
            data_year = ""
            if i == "url_id_data":
                url = "https://raw.githubusercontent.com/Jon-Fr/Geo_419b/main/url_id_file.zip"
                file_name = "url_id_file.csv"
            else:
                url = """https://geoportal.geoportal-th.de/hoehendaten/Uebersichten/Stand_{}.zip""".format(url_year)
            if type_to_download == "auxiliary_data":
                hy_file_path = """image_data/{}/""".format(type_to_download)
            else:
                hy_file_path = """elevation_data/{}/""".format(type_to_download)
            if url_year == "2010-2013":
                file_name = "DGM2_2010-2013_Erfass-lt-Meta_UTM32-UTM_2014-12-10.shp"
            elif url_year == "2014-2019":
                file_name = "DGM1_2014-2019_Erfass-lt-Meta_UTM_2020-04-20--17127.shp"
            elif url_year == "2020-2025":
                file_name = "DGM1_2020-2025_Erfass-lt-Meta_UTM_2021-03--17127"
        if type_to_download == "dgm":
            url = """https://geoportal.geoportal-th.de/hoehendaten/DGM/dgm_{}/dgm{}_{}_1_th_{}.zip""" \
                .format(url_year, dem_n, i, url_year)
            data_kind = "dgm_"
            file_name = url[len(url) - 32:len(url) - 3] + "xyz"
            if year == 2020:
                file_name = url[len(url) - 35:len(url) - 3] + "xyz"
        if type_to_download == "dom":
            url = """http://geoportal.geoportal-th.de/hoehendaten/DOM/dom_{}/dom{}_{}_1_th_{}.zip""" \
                .format(url_year, dem_n, i, url_year)
            data_kind = "dom_"
            file_name = url[len(url) - 32:len(url) - 3] + "xyz"
            if year == 2020:
                file_name = url[len(url) - 35:len(url) - 3] + "xyz"
        if type_to_download == "las":
            url = """http://geoportal.geoportal-th.de/hoehendaten/LAS/las_{}/las_{}_1_th_{}.zip""" \
                .format(url_year, i, url_year)
            data_kind = "las_"
            file_name = url[len(url) - 32:len(url) - 3] + "laz"
            if year == 2020:
                file_name = url[len(url) - 35:len(url) - 3] + "laz"
        if type_to_download == "ortho":
            data_year = ""
            i = str(i)
            url = """https://geoportal.geoportal-th.de/gaialight-th/_apps/dladownload/download.php?type=op&id={}""" \
                .format(i)
            hy_file_path = """image_data/orthophotos/{}""".format(str(year_list[index]))
            file_name_part_1 = tile_number_list[index]
            file_name_part_2 = str(year_list[index])
        # set the name of the zip file
        zip_name = data_kind + i + data_year + ".zip"
        if type_to_download == "ortho":
            zip_name = "orthophoto_" + tile_number_list[index] + "_" + str(year_list[index]) + ".zip"
            index = index + 1
        # append the zip file name to zip file name list
        zip_data_list.append(zip_name[0:len(zip_name) - 4])
        # download the zip data file if there is no file with the name that it would get and
        # if the content of the zip file is not already present
        if not os.path.exists(zip_name) and \
                (not os.path.exists(hy_file_path + file_name) or type_to_download == "ortho"):
            # extra check for orthophotos (necessary because the full filename is harder to predict/construct)
            stop = False
            if type_to_download == "ortho" and os.path.exists(hy_file_path):
                file_list = os.listdir(hy_file_path)
                for file in file_list:
                    if file_name_part_1 in file and file_name_part_2 in file:
                        stop = True
                        break
            if stop is True:
                continue
            # set variables for the loop
            response = requests.get(url, stream=True)
            data = open(zip_name, "wb")
            # download and write data (downloading the data file in chunks is useful to save ram)
            for chunk in response.iter_content(chunk_size=1024):
                data.write(chunk)
            # close data
            data.close()
    # return the zip file name list if it is not empty
    if len(zip_data_list) > 0:
        return zip_data_list
    else:
        return "no_new_data"


meta_data_name = data_download(data_list_to_download=["meta_data_elevation_data_" + url_year],
                                                   year=year, type_to_download="meta_data", url_year=url_year)
zip_files_to_delete.append("meta_data_elevation_data_" + url_year)
print(meta_data_name)



# Create new folder and unzip metadata in it

In [ ]:
def create_and_unzip(folder_path, zip_files):
    # create folder
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    # loop through the list
    for zip_file in zip_files:
        # add .zip if necessary
        if ".zip" not in zip_file:
            zip_file = zip_file + ".zip"
        # check if file exist before trying to unzip it
        if os.path.exists(zip_file):
            # unzip the file
            zipped_data = zipfile.ZipFile(zip_file, "r")
            zipped_data.extractall(path=folder_path)
            zipped_data.close()

create_and_unzip(folder_path="elevation_data/meta_data", zip_files=["meta_data_elevation_data_" + url_year])

# Intersect GeoDataFrames
Intersects the GeoDataFrames of the metadata shapefile and the aoi

In [ ]:
def intersect_geodfs(geodf_1, geodf_2):
    # re-project geodf_1 if necessary
    re = geodf_2.crs == geodf_1.crs
    if re is False:
        geodf_1 = geodf_1.to_crs(geodf_2.crs)
    # execute join / intersect
    intersected_geodf = geopandas.sjoin(geodf_1, geodf_2, how="inner")
    # return the result
    return intersected_geodf

elev_meta_data_geodf = geopandas.read_file("elevation_data/meta_data/" + elev_meta_file)
aoi = geopandas.read_file(shp)
elev_meta_data_aoi = intersect_geodfs(geodf_1=aoi, geodf_2=elev_meta_data_geodf)

# Download list
Creates a list with all files to download

In [ ]:
def create_elev_download_list_simplified(elev_aoi,year):
    # month list
    m_l = ["-01", "-02", "-03", "-04", "-05", "-06", "-07", "-08", "-09", "-10", "-11", "-12"]
    year = str(year)
    filtered_data = elev_aoi[(elev_aoi["ERFASSUNG"] == year + m_l[0]) | (elev_aoi["ERFASSUNG"] == year + m_l[1]) |
                             (elev_aoi["ERFASSUNG"] == year + m_l[2]) | (elev_aoi["ERFASSUNG"] == year + m_l[3]) |
                             (elev_aoi["ERFASSUNG"] == year + m_l[4]) | (elev_aoi["ERFASSUNG"] == year + m_l[5]) |
                             (elev_aoi["ERFASSUNG"] == year + m_l[6]) | (elev_aoi["ERFASSUNG"] == year + m_l[7]) |
                             (elev_aoi["ERFASSUNG"] == year + m_l[8]) | (elev_aoi["ERFASSUNG"] == year + m_l[9]) |
                             (elev_aoi["ERFASSUNG"] == year + m_l[10]) | (elev_aoi["ERFASSUNG"] == year + m_l[11])]


    elev_download_list = list(filtered_data["NAME"])
    # return the list of relevant tiles
    return elev_download_list

elev_download_list = create_elev_download_list_simplified(elev_meta_data_aoi,year)
print(elev_download_list)

# Download dtm

In [ ]:
elev_data_list = data_download(type_to_download="dgm", url_year=url_year, year=year,
                                                       data_list_to_download=elev_download_list, dem_n=dem_n)
create_and_unzip(folder_path="elevation_data/dgm/" + str(year), zip_files=elev_data_list)
zip_files_to_delete.extend(elev_data_list)
print(elev_data_list)

# Delete zip-files 
They are unnecessary now

In [ ]:
def delete_zip_files(zip_files):
    # for loop to cycle through the whole list
    for i in zip_files:
        # check if the zip file to delete actually exist
        if os.path.exists(i + ".zip"):
            # delete the zip file
            os.remove(i + ".zip")

delete_zip_files(zip_files=zip_files_to_delete)

# Raster correction 

In [ ]:
def go_through_all_raster_simplified(dir, ending, file_cor=None):
    file_list = os.listdir(dir)
    out_file_list = []
    for i in range(len(file_list)):
        if file_list[i].endswith(ending):
            out_file_list.append(raster_correction(dir, file_list[i], file_cor, ending))
    return out_file_list

def raster_correction(dir, file_raster, file_cor, ending, epsg="EPSG: 25832"):
    raster_str = dir + "/" + file_raster
    out_file = raster_str.replace(ending, "_UTM_cor.tif")
    # open raster
    raster = gdal.Open(raster_str)
    gt = raster.GetGeoTransform()
    x_size = raster.RasterXSize
    y_size = raster.RasterYSize
    # calculate extension
    extent = [gt[0], gt[3] + gt[5] * y_size,
              gt[0] + gt[1] * x_size, gt[3]]
    # [minX, minY, maxX, maxY]

    # warp correction, so that it matches the input raster in crs, extension and resolution
    cor_warp = gdal.Warp("",
                         file_cor,
                         dstSRS=epsg,
                         xRes=gt[1],
                         yRes=gt[5],
                         resampleAlg='bilinear',
                         outputBounds=extent,
                         format="vrt")
    # correction
    data_out = raster.GetRasterBand(1).ReadAsArray() + cor_warp.GetRasterBand(1).ReadAsArray()
    # driver for output
    driver = gdal.GetDriverByName("GTiff")
    ds_out = driver.Create(out_file, x_size, y_size, 1, gdal.GDT_UInt16)
    ds_out.SetGeoTransform(cor_warp.GetGeoTransform())  # sets same geotransform as input
    ds_out.SetProjection(cor_warp.GetProjection())  # sets same projection as input
    band_out = ds_out.GetRasterBand(1)
    band_out.WriteArray(data_out)
    return {"file": out_file, "extent": extent}

dgm_list = go_through_all_raster_simplified("./elevation_data/dgm/"+str(year), ".xyz", file_cor_dgm)
print(dgm_list)

# Raster mosaic
Merging all downloaded raster together to a mosaic

In [ ]:
class GeoFileHandler:
    def __init__(self, path, folder_name, geo_file_list):
        self.folder = path + "/" + folder_name
        self.name = folder_name
        self.geo_file_list = geo_file_list
        full_extent = []
        self.file_list = []
        for i in range(len(geo_file_list)):
            self.file_list.append(geo_file_list[i]["file"])
            # needed for gdal.BuildVRT() in method create_vrt
            if i == 0:
                # extent of first raster is full extent temporary
                full_extent = geo_file_list[i]["extent"]
            else:
                for key in [0, 1]:
                    # if new minimum
                    if geo_file_list[i]["extent"][key] < full_extent[key]:
                        full_extent[key] = geo_file_list[i]["extent"][key]
                for key in [2, 3]:
                    # if new maximum
                    if geo_file_list[i]["extent"][key] > full_extent[key]:
                        full_extent[key] = geo_file_list[i]["extent"][key]
        self.extent = full_extent

    def create_vrt(self, name, epsg="EPSG: 25832"):
        # build vrt
        opts = gdal.BuildVRTOptions(outputBounds=self.extent)
        vrt = gdal.BuildVRT(self.folder + "/"+name+".vrt", self.file_list, options=opts)
        # warp in chosen crs
        vrt_warped = gdal.Warp("", vrt, dstSRS=epsg, format='vrt')
        # write as GeoTiff
        gdal.Translate(self.folder + "/"+name+".tif", vrt_warped, format='GTiff',
                       creationOptions=['COMPRESS:DEFLATE', 'TILED:YES'])

handler = GeoFileHandler("./elevation_data/dgm/", str(year), dgm_list)
handler.create_vrt(name="mosaic_dgm_"+str(year),epsg="EPSG: 4326")

# Visualization of the results

In [ ]:
import folium
from folium import plugins
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es

mosaic_file = "./elevation_data/dgm/"+str(year)+"/mosaic_dgm_"+str(year)+".tif"

raster = gdal.Open(mosaic_file)
gt = raster.GetGeoTransform()
x_size = raster.RasterXSize
y_size = raster.RasterYSize
# calculate extension
extent = [gt[0], gt[3] + gt[5] * y_size,
          gt[0] + gt[1] * x_size, gt[3]]
# [minX, minY, maxX, maxY]
print(extent)

m = folium.Map(location=[(extent[1]+extent[3])/2,(extent[0]+extent[2])/2])


# Open the raster in rioxarray
img = rxr.open_rasterio(mosaic_file, masked=True)

# Reproject the raster to be the correct crs
#img = img.rio.reproject('EPSG:4326')

# Replace all null values with the minimum value in the array
img_plot = img.where(~img.isnull(), img.min())

# Scale the array from 0 to 255
scaled_img = es.bytescale(img_plot.values[0])

map_bounds = [[extent[1], extent[0]], 
              [extent[3], extent[2]]]

m.add_child(folium.raster_layers.ImageOverlay(scaled_img, 
                                              opacity=.9,
                                              bounds=map_bounds))

# Display the map
m